# Relative clauses
## Author: Kyle Rawlins

This notebook goes through the analysis of relative clauses in Heim & Kratzer (1998), "Semantics in generative grammar".

In [ ]:
reload_lamb()
composition_system = lang.hk3_system.copy()
lang.set_system(composition_system)
composition_system

In [ ]:
%%lamb reset
||gray|| = lambda x_e : Gray(x)
||cat|| = lambda x_e : Cat(x)
||bill|| = Bill_e
||likes|| = lambda x_e : lambda y_e : Likes(y,x)

In [ ]:
gray * cat

What is needed for variable binding?  This takes two parts:

 * specialized Item classes for the binder and the traces/variables.
 * code to actually perform predicate abstraction.
 
Both of these are provided as part of `lamb.lang`, in the form of the classes `Binder`, `Trace`, and `IndexedPronoun`, as well as several version of predicate abstraction.  The current composition system is using `lamb.lang.tree_pa_metalanguage_fun`, if you are curious.  (Side note: PA rules in the lambda notebook may not work as you expect; they operate by using variables named with a particular syntax and assignment functions in the metalanguage, rather than assignments for the object language.)

Now let's use these things.  Binders and traces can easily be constructed around numeric indices, and used anywhere you would use a regular lexical item:

In [ ]:
b = lang.Binder(5)
t = lang.Trace(5)
t

In [ ]:
(b * t).paths()

In [ ]:
bill * (likes * t)

A simple relative clause like "that Bill likes" can be represented as follows:

In [ ]:
r = b * (bill * (likes * t))
r

This would compose with a NP via Predicate Modification.

In [ ]:
r = gray * (cat * (b * (bill * (likes * t))))
r

In [ ]:
r.paths()

Now for a very complicated example.  This is one of the more challenging examples from Heim and Kratzer, the such-that relative "the man such that Mary reviewed a book that he wrote".

In [ ]:
he = lang.IndexedPronoun("he", 2, types.type_e)
he

In [ ]:
%%lamb
||mary|| = Mary_e
||man|| = L x_e : Man(x)
||book|| = L x_e : Book(x)
||wrote|| = L x_e : L y_e : Wrote(y,x)
||reviewed|| = L x_e : L y_e : Reviewed(y,x)
||that|| = L p_t : p
||such|| = L p_t : p
||a|| = L f_<e,t> : L g_<e,<e,t>> : L y_e : Exists x_e : f(x) & g(x)(y)

In [ ]:
(lang.Binder(2) * (such * (that * (mary * (reviewed * (a * (book * (lang.Binder(1) * (he * (wrote * lang.Trace(1)))))))))))

In [ ]:
r = man * (lang.Binder(2) * (such * (that * (mary * (reviewed * (a * (book * (lang.Binder(1) * (he * (wrote * lang.Trace(1)))))))))))
r.paths()

### Using presuppositional pronouns

The partiality calculation process in this case is somewhat buggy, resulting in redundant partiality conditions being multiplied.

In [ ]:
from lamb.meta import Partial
lang.set_system(lang.td_presup)
lang.get_system()

In [ ]:
male_prop = %te L x_e : Male_<e,t>(x)
hei = lang.PresupPronoun.index_factory("he", male_prop)
hei(2)

In [ ]:
%%lamb
||mary|| = Mary_e
||man|| = L x_e : Man(x)
||book|| = L x_e : Book(x)
||wrote|| = L x_e : L y_e : Wrote(y,x)
||reviewed|| = L x_e : L y_e : Reviewed(y,x)
||that|| = L p_t : p
||such|| = L p_t : p
||a|| = L f_<e,t> : L g_<e,<e,t>> : L y_e : Exists x_e : f(x) & g(x)(y)
||the|| = lambda f_<e,t> : Iota x : f(x)

In [ ]:
r = man * (lang.Binder(2) * (such * (that * (mary * (reviewed * (a * (book * (lang.Binder(1) * (hei(2) * (wrote * lang.Trace(1)))))))))))
the * r